# Création de fichiers CSV pour la future base de données FAO

### En vue de l'insertion des données dans une base de données relationnelle

In [1]:
# Création des différentes tables (nettoyage données, mise en forme...) 
# Export en CSV
# Création d'une BDD avec incorporation des tables

In [2]:
# coding: utf8

In [3]:
import pandas as pd

In [4]:
import numpy as np

In [5]:
# -*- coding: utf-8 -*-
import sqlite3

In [6]:
# Code to find primary key (with 1 or multiple columns)
def is_primary_key(df, cols_to_check):
    '''
    cols_to_check: (List)
    '''
    for col in cols_to_check :
        if col not in cols_to_check :
            raise ValueError('These columns are not in the dataframe.')
    if len(df) != len(df[cols_to_check].drop_duplicates()):
        raise Exception("The set of columns: {}, can't define a primary key".format(cols_to_check))
    print('Primary key found!')

In [7]:
# Filtre sur la colonne 'element' (avant pivot)
def filtre_element(code_pays,intitule_element,code_produit, df):
    pays_filter = df['code_pays'] == code_pays
    elementFilter = df['element'] == intitule_element
    productFilter = df['code_produit'] == code_produit
    df_selection = df[pays_filter & elementFilter & productFilter]
    return df_selection
   

In [8]:
# Création de la table 'population' contenant la population de chaque pays pour 2013. 
# Elle contient 4 colonnes : pays, code_pays, annee, population.

In [9]:
populationCsv = pd.read_csv("fr_population.csv", encoding='UTF-8')

In [10]:
# Sélection des colonnes Zone, Code zone, Année, Produit
populationForm = populationCsv.loc[:, ['Code zone', 'Zone', 'Valeur', 'Année']]

In [11]:
# 'Valeur' correspond au nombre d'individus (=> * 1000)
populationForm['Valeur'] = populationForm['Valeur'] * 1000

In [12]:
# Renommage des colomnes
populationRen = populationForm.rename({'Code zone':'code_pays',
                                'Zone': 'pays',
                                'Valeur':'population',
                                'Année':'annee'}, axis=1)                            
                        

In [13]:
# Les colomnes sont mises dans l'ordre : pays, code_pays, annee, population
populationRen = populationRen[['pays', 'code_pays', 'annee', 'population']]

In [14]:
# Suppression de la Chine :
population = populationRen[~populationRen.pays.isin(['Chine'])]

In [15]:
population.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 174 entries, 0 to 174
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   pays        174 non-null    object
 1   code_pays   174 non-null    int64 
 2   annee       174 non-null    int64 
 3   population  174 non-null    int64 
dtypes: int64(3), object(1)
memory usage: 6.8+ KB


In [16]:
type(population)

pandas.core.frame.DataFrame

In [17]:
population.shape

(174, 4)

## Question 15 : Proposez une clé primaire pertinente pour la table 'population.

In [18]:
# Clé primaire : pays - code_pays

In [19]:
population.columns

Index(['pays', 'code_pays', 'annee', 'population'], dtype='object')

In [20]:
is_primary_key(population, ['code_pays'])

Primary key found!


In [21]:
# Export de la table population en csv
population.to_csv("population.csv", index = False, encoding='UTF-8')

In [22]:
# Création d'une table 'dispo_alim' contenant pour chaque pays et pour chaque produit en 2013, les informations suivantes:
# la nature du produit (deux valeurs possibles : “animal” ou “végétal”)
# disponibilité alimentaire en tonnes
# disponibilité alimentaire en Kcal/personne/jour
# disponibilité alimentaire de protéines en g/personne/jour
# disponibilité alimentaire de matières grasses en g/personne/jour
# Elle contient les colonnes : pays, code_pays, année, produit, code_produit, origin, dispo_alim_tonnes, dispo_alim_kcal_p_j, dispo_prot, dispo_mat_gr .


In [23]:
df_animaux = pd.read_csv("fr_animaux.csv")
df_vegetaux = pd.read_csv("fr_vegetaux.csv")

In [24]:
# Ajout d'une colonne 'nature' contenant 'animal' pour 'df_animaux'
df_animaux['origin'] = 'animal'

In [25]:
# Ajout d'une colonne 'nature' contenant 'végétal' pour 'df_vegetaux'
df_vegetaux['origin'] = 'végétal'

In [26]:
dispo_alimForm = pd.concat([df_animaux, df_vegetaux])

In [27]:
# Recherche des occurences => aucune occurence
dispo_alimForm.duplicated(subset=['Zone','Élément', 'Produit', ]).sum()

0

In [28]:
# Vérification de la présence de "NaN" => dataframe sain
dispo_alimForm.isna().mean(axis=0)

Code Domaine              0.0
Domaine                   0.0
Code zone                 0.0
Zone                      0.0
Code Élément              0.0
Élément                   0.0
Code Produit              0.0
Produit                   0.0
Code année                0.0
Année                     0.0
Unité                     0.0
Valeur                    0.0
Symbole                   0.0
Description du Symbole    0.0
origin                    0.0
dtype: float64

In [29]:
dispo_alimForm.shape

(142037, 15)

In [30]:
dispo_alimForm['Élément'].unique()

array(['Production', 'Importations - Quantité',
       'Disponibilité intérieure', 'Nourriture',
       'Disponibilité alimentaire en quantité (kg/personne/an)',
       'Disponibilité alimentaire (Kcal/personne/jour)',
       'Disponibilité de protéines en quantité (g/personne/jour)',
       'Disponibilité de matière grasse en quantité (g/personne/jour)',
       'Semences', 'Pertes', 'Exportations - Quantité',
       'Aliments pour animaux', 'Autres utilisations (non alimentaire)',
       'Variation de stock', 'Traitement'], dtype=object)

In [31]:
# Suppression de la Chine:
dispo_alimForm = dispo_alimForm[~dispo_alimForm.Zone.isin(['Chine'])]

In [32]:
# Récupération des 'disponibilité alimentaire' de la colonne 'Élément'
elements_to_keep = ['Disponibilité alimentaire en quantité (kg/personne/an)', 'Disponibilité alimentaire (Kcal/personne/jour)',
                   'Disponibilité de protéines en quantité (g/personne/jour)', 'Disponibilité de matière grasse en quantité (g/personne/jour)']
elements_filter = dispo_alimForm['Élément'].isin(elements_to_keep)
dispo_alimFormDis = dispo_alimForm[elements_filter]

In [33]:
# Sélection des colonnes Zone, Code zone, Année, Produit
dispo_alimFormDis = dispo_alimFormDis.loc[:, ['Code zone', 'Zone', 'Élément', 'Code Produit', 'Produit', 'Année', 
                               'Valeur','origin']]

In [34]:
# Renommage des colonnes :
dispo_alimFormDis = dispo_alimFormDis.rename({'Code zone':'code_pays',                                
                                'Zone': 'pays',
                                'Élément': 'element',
                                'Code Produit' : 'code_produit',
                                'Produit' : 'produit',
                                'Année':'annee',                               
                                'Valeur' : 'Valeur',
                                'origin' : 'origin'
                               }, axis=1)  

In [35]:
#  Re indexage
dispo_alimFormDis = dispo_alimFormDis.reset_index(drop=True)

In [36]:
dispo_alimFormDis.dtypes

code_pays         int64
pays             object
element          object
code_produit      int64
produit          object
annee             int64
Valeur          float64
origin           object
dtype: object

In [37]:
# On applique un 'pivot-table' sur 'dispo_alimForm': Les items de 'Élément' sont sur une colonne
dispo_alimFormDisPiv = pd.pivot_table(dispo_alimFormDis, values='Valeur', index=['code_pays', 'pays','code_produit',
                                                                                 'produit', 'annee', 'origin'],
                                      columns='element', aggfunc='sum').reset_index()

In [38]:
# Recherche des NaN
dispo_alimFormDisPiv.isna().mean()

element
code_pays                                                        0.000000
pays                                                             0.000000
code_produit                                                     0.000000
produit                                                          0.000000
annee                                                            0.000000
origin                                                           0.000000
Disponibilité alimentaire (Kcal/personne/jour)                   0.001332
Disponibilité alimentaire en quantité (kg/personne/an)           0.017181
Disponibilité de matière grasse en quantité (g/personne/jour)    0.172931
Disponibilité de protéines en quantité (g/personne/jour)         0.189271
dtype: float64

In [39]:
dispo_alimFormDisPiv.shape

(14260, 10)

In [40]:
# Choix de ne pas supprimer les NaN car il y a trop de répercussion sur les résultats des recherches futures

In [41]:
dispo_alimFormDisPiv.shape

(14260, 10)

In [42]:
# Renommage des colomnes
dispo_alimFormDisPiv = dispo_alimFormDisPiv.rename({'Disponibilité alimentaire (Kcal/personne/jour)': 'dispo_alim_kcal_p_j',
                               'Disponibilité alimentaire en quantité (kg/personne/an)': 'Disponibilité alimentaire en quantité (kg/personne/an)',
                               'Disponibilité de matière grasse en quantité (g/personne/jour)': 'dispo_mat_gr',
                               'Disponibilité de protéines en quantité (g/personne/jour)': 'dispo_prot'}, axis=1)     

In [43]:
# Re indexage
dispo_alimFormDisPiv = dispo_alimFormDisPiv.reset_index(drop=True)

In [44]:
dispo_alimFormDisPiv.columns

Index(['code_pays', 'pays', 'code_produit', 'produit', 'annee', 'origin',
       'dispo_alim_kcal_p_j',
       'Disponibilité alimentaire en quantité (kg/personne/an)',
       'dispo_mat_gr', 'dispo_prot'],
      dtype='object', name='element')

In [45]:
# Pour avoir la disponibilité alimentaire en tonnes, il faut fusionner avec 'population'
dispo_alimFormDisPivPop = pd.merge(dispo_alimFormDisPiv, population)

In [46]:
dispo_alimFormDisPivPop.shape

(14260, 11)

In [47]:
dispo_alimFormDisPivPop.isna().mean()

code_pays                                                 0.000000
pays                                                      0.000000
code_produit                                              0.000000
produit                                                   0.000000
annee                                                     0.000000
origin                                                    0.000000
dispo_alim_kcal_p_j                                       0.001332
Disponibilité alimentaire en quantité (kg/personne/an)    0.017181
dispo_mat_gr                                              0.172931
dispo_prot                                                0.189271
population                                                0.000000
dtype: float64

In [48]:
# Création de la colonne 'dispo_alim_tonnes' => 'Disponibilité alimentaire en quantité (kg/personne/an)' * population /1000
dispo_alimFormDisPivPop['dispo_alim_tonnes'] = dispo_alimFormDisPivPop['Disponibilité alimentaire en quantité (kg/personne/an)'] * dispo_alimFormDisPivPop['population'] / 1000

In [49]:
dispo_alimFormDisPivPop.shape

(14260, 12)

In [50]:
dispo_alimFormDisPivPop.columns

Index(['code_pays', 'pays', 'code_produit', 'produit', 'annee', 'origin',
       'dispo_alim_kcal_p_j',
       'Disponibilité alimentaire en quantité (kg/personne/an)',
       'dispo_mat_gr', 'dispo_prot', 'population', 'dispo_alim_tonnes'],
      dtype='object')

In [51]:
# Les colomnes sont mises dans l'ordre : pays, code_pays, année, produit, code_produit, origin, dispo_alim_tonnes, dispo_alim_kcal_p_j, dispo_prot, dispo_mat_g
dispo_alim = dispo_alimFormDisPivPop[['pays', 'code_pays', 'annee', 'produit', 'code_produit', 
                                     'origin', 'dispo_alim_tonnes', 'dispo_alim_kcal_p_j', 
                                     'dispo_prot', 'dispo_mat_gr']]

In [52]:
dispo_alim.shape

(14260, 10)

## Question 16 : Proposez une clé primaire pertinente pour la table 'dispo_alim'

In [53]:
# Clé primaire : 'code_pays' - 'code_produit' -

In [54]:
is_primary_key(dispo_alim, ['code_pays','code_produit' ])

Primary key found!


In [55]:
# Export de 'dispo_alim' en csv
dispo_alim.to_csv("dispo_alim.csv", index = False)
# C:\Users\isabe\Documents\FAO_2013essai

In [56]:
# Création d'une table appelée 'equilibre_prod' contenant pour chaque pays et pour chaque produit en 2013, les quantités suivantes :
#         disponibilité intérieure
#         aliments pour animaux
#         semences
#         pertes
#         transformés
#         nourriture
#         autres utilisations
# Elle devra contenir ces colonnes : 
# pays, code_pays, année, produit, code_produit, dispo_int, alim_ani, semences, pertes, transfo, nourriture, autres_utilisations.


In [57]:
dispo_alimForm['Élément'].unique()

array(['Production', 'Importations - Quantité',
       'Disponibilité intérieure', 'Nourriture',
       'Disponibilité alimentaire en quantité (kg/personne/an)',
       'Disponibilité alimentaire (Kcal/personne/jour)',
       'Disponibilité de protéines en quantité (g/personne/jour)',
       'Disponibilité de matière grasse en quantité (g/personne/jour)',
       'Semences', 'Pertes', 'Exportations - Quantité',
       'Aliments pour animaux', 'Autres utilisations (non alimentaire)',
       'Variation de stock', 'Traitement'], dtype=object)

In [58]:
# Récupération des intitulés demandés de la colonne 'Élément'
elements_to_keep = ['Disponibilité intérieure', 'Aliments pour animaux','Semences', 'Pertes',
                    'Traitement', 'Nourriture', 'Autres utilisations (non alimentaire)'] 
elements_filter = dispo_alimForm['Élément'].isin(elements_to_keep)
equilibre_prodEqu = dispo_alimForm[elements_filter]

In [59]:
# Sélection des colonnes Zone, Code zone, Année, Produit
equilibre_prodEqu = equilibre_prodEqu.loc[:, ['Code zone', 'Zone', 'Élément', 'Code Produit', 'Produit', 'Année', 
                               'Valeur']]

In [60]:
# Renommage des colonnes :
equilibre_prodEqu = equilibre_prodEqu.rename({'Code zone':'code_pays',                                
                                'Zone': 'pays',
                                'Élément': 'element',
                                'Code Produit' : 'code_produit',
                                'Produit' : 'produit',
                                'Année':'annee',                               
                                'Valeur' : 'Valeur'                                
                               }, axis=1)  

In [61]:
# Juste un essai de fonction:
filtre_element(17, 'Nourriture', 2514, equilibre_prodEqu) 

,code_pays,pays,element,code_produit,produit,annee,Valeur
12186,17,Bermudes,Nourriture,2514,Maïs,2013,0.0


In [62]:
# Re indexage
equilibre_prodEqu = equilibre_prodEqu.reset_index(drop=True)

In [63]:
equilibre_prodEqu.columns

Index(['code_pays', 'pays', 'element', 'code_produit', 'produit', 'annee',
       'Valeur'],
      dtype='object')

In [64]:
# On applique un 'pivot-table' sur 'equilibre_prodEqu': Élément devient colonne
equilibre_prodEquPiv = pd.pivot_table(equilibre_prodEqu, values='Valeur', index=['code_pays', 'pays','code_produit', 'produit', 'annee'], columns='element', aggfunc='sum').reset_index()

In [65]:
equilibre_prodEquPiv.columns

Index(['code_pays', 'pays', 'code_produit', 'produit', 'annee',
       'Aliments pour animaux', 'Autres utilisations (non alimentaire)',
       'Disponibilité intérieure', 'Nourriture', 'Pertes', 'Semences',
       'Traitement'],
      dtype='object', name='element')

In [66]:
equilibre_prodEquPiv.index

RangeIndex(start=0, stop=15382, step=1)

In [67]:
equilibre_prodEquPiv.shape

(15382, 12)

In [68]:
# Recherche des NaN :
equilibre_prodEquPiv.isna().mean()

element
code_pays                                0.000000
pays                                     0.000000
code_produit                             0.000000
produit                                  0.000000
annee                                    0.000000
Aliments pour animaux                    0.823170
Autres utilisations (non alimentaire)    0.642699
Disponibilité intérieure                 0.000000
Nourriture                               0.088870
Pertes                                   0.721883
Semences                                 0.864062
Traitement                               0.850995
dtype: float64

In [69]:
# Compte tenu du pourcentage élevé de NaN, Je fais le choix de garder les données NaN et de les remplacer par 0
equilibre_prodEquPivNa = equilibre_prodEquPiv.fillna(0, axis=0)

In [70]:
equilibre_prodEquPivNa.shape

(15382, 12)

In [71]:
equilibre_prodEquPivNa.columns

Index(['code_pays', 'pays', 'code_produit', 'produit', 'annee',
       'Aliments pour animaux', 'Autres utilisations (non alimentaire)',
       'Disponibilité intérieure', 'Nourriture', 'Pertes', 'Semences',
       'Traitement'],
      dtype='object', name='element')

In [72]:
# Renommage des colonnes :
equilibre_prodEquPivNa = equilibre_prodEquPivNa.rename({'code_pays':'code_pays',                                
                                'pays': 'pays',                                
                                'code_produit' : 'code_produit',
                                'produit' : 'produit',
                                'année' : 'annee',                               
                                'Aliments pour animaux' : 'alim_ani',
                                'Autres utilisations (non alimentaire)' : 'autres_utilisations',
                                'Disponibilité intérieure' : 'dispo_int',
                                'Nourriture' : 'nourriture',
                                'Pertes' : 'pertes',
                                'Semences' : 'semences',
                                'Traitement' : 'transfo'}, axis=1)  

In [73]:
# Les colomnes sont mises dans l'ordre : pays, code_pays, année, produit, code_produit, dispo_int, alim_ani, semences, pertes, transfo, nourriture, autres_utilisations.
equilibre_prod = equilibre_prodEquPivNa[['pays', 'code_pays', 'annee', 'produit', 'code_produit', 'dispo_int', 'alim_ani',
                               'semences', 'pertes', 'transfo', 'nourriture', 'autres_utilisations']]

## Question 17 : Proposez une clé primaire pertinente pour la table 'equilibre_prod' .

In [74]:
# Clé primaire : 'code_pays' - 'code_produit' 

In [75]:
is_primary_key(equilibre_prod, ['code_pays', 'code_produit'])

Primary key found!


In [76]:
# Export de la table equilibre_prod en csv
equilibre_prod.to_csv("equilibre_prod.csv", index = False)

In [77]:
# Création d'une table 'sous_nutrition', contenant :
# le nombre de personnes en sous-alimentation pour chaque pays en 2013. 
# Elle devra contenir 4 colonnes : pays, code_pays, année, nb_personnes.

In [78]:
# Importation du dataframe
# Nettoyage des données (changement de type...)
# Harmonisation des données (dates)

In [79]:
# Importation du dataframe : 
sous_nutriCsv = pd.read_csv('fr_sousalimentation.csv', encoding='UTF-8')

In [80]:
sous_nutriCsv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1020 entries, 0 to 1019
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Code Domaine            1020 non-null   object 
 1   Domaine                 1020 non-null   object 
 2   Code zone               1020 non-null   int64  
 3   Zone                    1020 non-null   object 
 4   Code Élément            1020 non-null   int64  
 5   Élément                 1020 non-null   object 
 6   Code Produit            1020 non-null   int64  
 7   Produit                 1020 non-null   object 
 8   Code année              1020 non-null   int64  
 9   Année                   1020 non-null   object 
 10  Unité                   1020 non-null   object 
 11  Valeur                  605 non-null    object 
 12  Symbole                 1020 non-null   object 
 13  Description du Symbole  1020 non-null   object 
 14  Note                    0 non-null      

In [81]:
# Conversion de la colonne 'Valeur' (de type 'objet') en type float
sous_nutriCsv.Valeur = pd.to_numeric(sous_nutriCsv.Valeur, errors='coerce')

In [82]:
sous_nutriCsv.Valeur.dtypes

dtype('float64')

In [83]:
sous_nutriCsv.shape

(1020, 15)

In [84]:
# Sélection des colonnes Zone, Code zone, Année, Produit
sous_nutriForm = sous_nutriCsv.loc[:, ['Code zone', 'Zone', 'Valeur', 'Année']]

In [85]:
# Suppression de la Chine :
sous_nutriForm = sous_nutriForm[~sous_nutriForm.Zone.isin(['Chine'])]

In [86]:
# Renommage des colonnes
sous_nutriForm = sous_nutriForm.rename({'Code zone':'code_pays',
                                'Zone': 'pays',
                                'Valeur':'nb_personnes',
                                'Année':'annee'}, axis=1)   

In [87]:
# Ré indexage
sous_nutriForm = sous_nutriForm.reset_index(drop=True)

In [88]:
# Modification de la colonne 'nb_personnes' pour avoir le nombre réel d'individus (=> * 1000000) :
sous_nutriForm.nb_personnes = sous_nutriForm.nb_personnes * 1000000

In [89]:
sous_nutriForm['annee'].value_counts()

2013-2015    203
2015-2017    203
2014-2016    203
2016-2018    203
2012-2014    203
Name: annee, dtype: int64

In [90]:
# Les données de la colonne 'année' sont harmonisées pour être au même format que celles des autres dataframes
sous_nutriForm['annee'] = sous_nutriForm['annee'].map({'2012-2014' : 2013,'2013-2015' : 2014, '2014-2016' : 2015, '2015-2017' : 2016, '2016-2018' : 2017}) 

In [91]:
# Filtre sur l'année '2013' :
sous_nutriFormFao = sous_nutriForm[sous_nutriForm['annee'] == 2013]

In [92]:
sous_nutriFormFao.shape

(203, 4)

In [93]:
sous_nutriFormFao.describe()

,code_pays,nb_personnes,annee
count,203.000000,9.600000e+01,203.0
mean,128.172414,7.746875e+06,2013.0
std,75.349646,2.615793e+07,0.0
min,1.000000,1.000000e+05,2013.0
25%,62.000000,5.000000e+05,2013.0
50%,127.000000,1.750000e+06,2013.0
75%,192.000000,5.375000e+06,2013.0
max,299.000000,2.163000e+08,2013.0


In [94]:
# Recherche des NaN : 
sous_nutriFormFao.isna().mean()

code_pays       0.000000
pays            0.000000
nb_personnes    0.527094
annee           0.000000
dtype: float64

In [95]:
# Je fais le choix de perdre des données plutôt que de modifier les données
sous_nutriFormFaoNa = sous_nutriFormFao.dropna(axis=0)

In [96]:
# Ré indexage
sous_nutriFormFaoNa = sous_nutriFormFaoNa.reset_index(drop=True)

In [97]:
# Recherche des valeurs dupliquées 
sous_nutriFormFaoNa.duplicated(subset=['code_pays']).sum()

0

In [98]:
# Les colomnes sont mises dans l'ordre : pays, code_pays, année, nb_personnes
sous_nutrition = sous_nutriFormFaoNa[['pays', 'code_pays', 'annee', 'nb_personnes']]

## Question 18 : Proposez une clé primaire pertinente pour la table sous_nutrition.

In [99]:
# Clé primaire : 'code_pays'

In [100]:
is_primary_key(sous_nutrition, ['code_pays'])

Primary key found!


In [101]:
# Export de 'sous_nutrition' en csv
sous_nutrition.to_csv("sous_nutrition.csv", index = False)
# C:\Users\isabe\Documents\FAO_2013essai